<center>
<img src="../../img/ods_stickers.jpg"/> 
    
### <center> Author: Rajasekhar Battula, @Rajasekhar Battula
    
## <center> Tutorial
### <center> Cricket Scorecard Data Scraping.

Do you like Cricket? I think you do.

In this tutorial we will discuss the way to convert publicly available cricket scorecard of any cricket match from the website (www.espncricinfo.com) to Structured tabular format in python dataframe. For the same purpose I'd make use of a python library espncricinfo.

 For each cricket there is a unique MatchID provided at the end of a match page URL in the website ('http://www.espncricinfo.com/matches/engine/match/1119496.html'). MatchID is highlighted with red box in the URL. We will use this ID to fetch score card details. Lets use of couple of matches to explain in this tutorial.
1. Match 1: India vs Australia, Sep 17 2017  --> MatchID = '**1119496**'
2. Match 2: England vs New Zealand, Jun 6 2017   --> MatchID = '**1022357**'

<img src="../../img/Cricket_1.png"/>

In [2]:
#Install the required libraries as below
!pip install python-espncricinfo
!pip install grequests

In [3]:
from espncricinfo.summary import Summary
from espncricinfo.match import Match 
from espncricinfo.series import Series

import json
import requests
from bs4 import BeautifulSoup
from espncricinfo.exceptions import MatchNotFoundError, NoScorecardError

import pandas as pd

C:\Users\rbat\AppData\Local\Continuum\anaconda3\lib\site-packages\grequests.py:21: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util (C:\\Users\\rbat\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\urllib3\\util\\__init__.py)', 'urllib3.contrib.pyopenssl (C:\\Users\\rbat\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\urllib3\\contrib\\pyopenssl.py)']. 
  curious_george.patch_all(thread=False, select=False)
C:\Users\rbat\AppData\Local\Continuum\anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


For the sake of illustration i've used the below couple of matchID's. We can use any number of them and it depends on your system computational power.

In [4]:
testlist = ['1119496', '1022357']

In [5]:
#To see the match URL we can use the below function within espncricinfo library
Match('1119496').match_url

'http://www.espncricinfo.com/matches/engine/match/1119496.html'

In [6]:
#This functions helps to expand the list of dictonaries to columns in a dataframe.
def flatten(js):
    return pd.DataFrame(js).set_index(['text','name']).squeeze()

Steps to be followed to create the structured data from ESPNCricinfo website:
-  To extract the scorecard details we create 2 dataframes one for batsmen and other for bowllers
-  Firstly, extract the HTML content of the match using the Match(<matchID>).html
-  Locate the scorecard script in the HTML content using the method "find_all" of BeautifulSoup.
-  Convert it to JSON format.
-  Extract the scorecard of each batsmen or bowller using the key-value pair of JSON dicts.
-  Flatten the scorecard of each batsmen or bowller using the "flatten" function written above.
-  Using the helper functions in espncricinfo library extract the city and date of match.
-  Finally return a dataframe with all the above details.

In [7]:
def getbattingdatafame(list1):
    df = pd.DataFrame()
    for x in list1:
        x1 = Match(x).html
        x2 = json.loads(x1.find_all('script')[12].get_text().replace("\n", " ").replace('window.__INITIAL_STATE__ =','').replace('&dagger;','wk').replace('&amp;','').replace('wkts;','wkts,').replace('wkt;','wkt,').strip().replace('};', "}};").split('};')[0])
        df1bat = pd.DataFrame(x2['gamePackage']['scorecard']['innings']['1']['batsmen'])
        d1title = x2['gamePackage']['scorecard']['innings']['1']['title']
        df1bat['Team'] = d1title.split(' ')[0]
        df2bat = pd.DataFrame(x2['gamePackage']['scorecard']['innings']['2']['batsmen'])
        d2title = x2['gamePackage']['scorecard']['innings']['2']['title']
        df2bat['Team'] = d2title.split(' ')[0]
        df1bat['Oppositionteam'] = d2title.split(' ')[0]
        df2bat['Oppositionteam'] = d1title.split(' ')[0]
        
        Finaldf_bat = pd.concat([df1bat.drop(['captain','commentary','runningScore','runningOver', 'stats','hasVideoId','href','isNotOut','roles','shortText','trackingName'], axis=1),
           df1bat.stats.apply(flatten)], axis=1).append(pd.concat([df2bat.drop(['captain','commentary','runningScore','runningOver', 'stats','hasVideoId','href','isNotOut','roles','shortText','trackingName'], axis=1),
                                                               df2bat.stats.apply(flatten)], axis=1))
        Finaldf_bat['city'] = Match(x).town_name
        Finaldf_bat['date'] = Match(x).date
        df=pd.concat([df,Finaldf_bat])
    return(df)

In [8]:
getbattingdatafame(testlist).head()

,name,Team,Oppositionteam,"(R, runs)","(B, ballsFaced)","(M, minutes)","(4s, fours)","(6s, sixes)","(SR, strikeRate)",city,date
0,AM Rahane,India,Australia,5,15,18,0,0,33.33,Chennai,2017-09-17
1,RG Sharma,India,Australia,28,44,81,3,0,63.63,Chennai,2017-09-17
2,V Kohli,India,Australia,0,4,7,0,0,0.00,Chennai,2017-09-17
3,MK Pandey,India,Australia,0,2,4,0,0,0.00,Chennai,2017-09-17
4,KM Jadhav,India,Australia,40,54,72,5,0,74.07,Chennai,2017-09-17


In [9]:
def getbowlingdatafame(list1):
    df = pd.DataFrame()
    for x in list1:
        x1 = Match(x).html
        x2 = json.loads(x1.find_all('script')[12].get_text().replace("\n", " ").replace('window.__INITIAL_STATE__ =','').replace('&dagger;','wk').replace('&amp;','').replace('wkts;','wkts,').replace('wkt;','wkt,').strip().replace('};', "}};").split('};')[0])
        df1bowl = pd.DataFrame(x2['gamePackage']['scorecard']['innings']['1']['bowlers'])
        d1title = x2['gamePackage']['scorecard']['innings']['1']['title']
        df2bowl = pd.DataFrame(x2['gamePackage']['scorecard']['innings']['2']['bowlers'])
        d2title = x2['gamePackage']['scorecard']['innings']['2']['title']
        df1bowl['Team'] = d2title.split(' ')[0]
        df2bowl['Team'] = d1title.split(' ')[0]
        df1bowl['Oppositionteam'] = d1title.split(' ')[0]
        df2bowl['Oppositionteam'] = d2title.split(' ')[0]
        
        Finaldf_bowl = pd.concat([df1bowl.drop(['captain','stats','hasVideoId','href','roles','trackingName'], axis=1),
                       df1bowl.stats.apply(flatten)], axis=1).append(pd.concat([df2bowl.drop(['captain','stats','hasVideoId','href','roles','trackingName'], axis=1),
                                                               df2bowl.stats.apply(flatten)], axis=1))
        Finaldf_bowl['city'] = Match(x).town_name
        Finaldf_bowl['date'] = Match(x).date
        df=pd.concat([df,Finaldf_bowl])
    return(df)

In [10]:
getbowlingdatafame(testlist).head()

,name,Team,Oppositionteam,"(O, overs)","(M, maidens)","(R, conceded)","(W, wickets)","(Econ, economyRate)","(0s, dots)","(4s, foursConceded)","(6s, sixesConceded)","(WD, wides)","(NB, noballs)",city,date
0,PJ Cummins,Australia,India,10,1,44,0,4.40,33,3,0,2,0,Chennai,2017-09-17
1,NM Coulter-Nile,Australia,India,10,0,44,3,4.40,38,5,0,1,1,Chennai,2017-09-17
2,JP Faulkner,Australia,India,10,1,67,1,6.70,31,8,2,0,2,Chennai,2017-09-17
3,MP Stoinis,Australia,India,10,0,54,2,5.40,27,3,1,2,0,Chennai,2017-09-17
4,A Zampa,Australia,India,10,0,66,1,6.60,26,3,4,0,0,Chennai,2017-09-17


We have other helper functions to see the details of the series as shown below. 

In [13]:
print(Series('18808').years)
print(Series('18808').url)
print(Series('18808').name)

['2019']
http://www.espn.in/cricket/series/_/id/18808/india-in-new-zealand-2018-19
India tour of New Zealand 2018/19


C:\Users\rbat\AppData\Local\Continuum\anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


## References:
1. [espncricinfo reference](https://github.com/dwillis/python-espncricinfo)<br>
2. [Scorecards from YAML files](https://github.com/tvganesh/yorkpy/blob/master/yorkpy/analytics.py)<br>